# Przed rozpoczęciem

* Stwórz nowy compute lub użyj istniejącego compute w wersji: `Standard_F16s_v2`
* Wybierz kernel `Python 3.8 - AzureML`

# Przygotowanie środowiska

* Ta komórka nie zawiera kodu w pythonie, tylko wywołania `pip` w shellu terminalowym
* Przeinstalowujemy `tensorflow`, które już jest obecne w kernelu `AzureML` w wersji, która nie będzie rzucać błędami
* Instalujemy następujące biblioteki:
    * `sentence-transformers` - na potrzeby embeddera
    * `qdrant-client` - do komunikacji z wektorową bazą danych
    * `openai` - do komunikacji z API OpenAI

In [1]:
!pip install tensorflow-cpu==2.16.1
!pip install tf-keras==2.16.0 --no-dependencies
!pip install sentence-transformers
!pip install ctransformers
!pip install qdrant-client
!pip install openai

  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.16.2 which is incompatible.
mlflow-skinny 2.15.0 requires importlib-metadata!=4.7.0,<8,>=3.7.0, but you have importlib-metadata 8.2.0 which is incompatible.
grpcio-tools 1.68.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.
azureml-automl-runtime 1.57.0 requires protobuf<=3.20.3, but you have protobuf 4.25.5 which is incompatible.
azureml-automl-dnn-nlp 1.57.0 requires torch==1.13.1, but you have torch 2.4.1 which is incompatible.
azureml-automl-dnn-nlp 1.57.0 req

# Lokalne ładowanie modelu

W tym miejscu ściągamy i ładujemy model https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca korzystając z backendu `pytorch` (stąd `framework='pt'`)

Jeśli notebook byłby odpalony na lokalnej maszynie (np. laptop) to model zostanie ściągnięty na lokalny dysk i będzie działać lokalnie.

In [2]:
from transformers import pipeline
pipe = pipeline("text-generation", model="Open-Orca/Mistral-7B-OpenOrca", framework="pt")

2024-11-25 08:26:07.837775: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Wrzucamy pierwsze zapytanie do modelu. Warto zwrócić uwagę, że do modelu wchodzi lista interakcji różnych ról w modelem, a w odpowiedzi dostajemy podobną strukturę wzbogaconą o wyjście z LLMu

In [3]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant', 'content': ' I am an AI-powered text-based'}]}]

## Funkcja pomocnicza do ładnego wyświetlania interakcji z modelem

In [4]:
from rich.console import Console
console = Console()
def pretty_print(output):
    for entry in output[0]["generated_text"]:
        role, text = entry["role"], entry["content"]
        console.rule(f"[bold red]{role}", style="bright_blue")
        console.print(text)

##### Korzystamy z funkcji do ładnego wyświetlania i explicite ustawiamy maksymalną liczbę generowanych tokenów

In [5]:
messages = [
    {"role": "user", "content": "Is water wet? Elaborate"},
]
pretty_print(pipe(messages, max_new_tokens=200))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

Is water wet? Elaborate

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Water is indeed wet. The term "wet" refers to the presence of water or other liquids on a surface or in contact 
with an object. When water comes into contact with a surface, it adheres to it and can be absorbed by the material,
leading to a wet state.

Water is a polar molecule, meaning that it has a positive and a negative end. This polarity allows water molecules 
to form hydrogen bonds with other water molecules, which in turn creates a cohesive force that holds the liquid 
together. This property of water is responsible for its ability to wet surfaces and dissolve various substances.

In everyday life, we often associate the term "wet" with the feeling of moisture on our skin or the dampness of an 
object. When we touch water, the water molecules spread out and adhere to our skin, making it feel wet. Similarly, 
when an object is submerged in water or exposed to

#### Generowanie przykładów z ustawioną temperaturą

In [6]:
messages = [
    {"role": "user", "content": "Is water wet? Elaborate"},
]
pretty_print(pipe(messages, do_sample=True, temperature=1.2, max_new_tokens=200))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

Is water wet? Elaborate

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Water is indeed wet. This concept may be seen as quite intuitive or simple, as it refers to water's ability to 
adhere to surfaces and materials, making them moist. In fact, the term "wet" often describes situations when 
substances or objects have absorbed moisture, such as water. Additionally, the wetness of an object can change 
based on varying factors such as the amount of additional moisture absorbed or the nature of the surface.

Water, being a polar molecule, readily binds to other polar molecules, which also contributes to wetness. This 
means because water and other polar ingredients such as our own skin or other surfaces are mutually attracted, 
water can spread out and make the surface wet. 

It's also important to note that wetness isn't just limited to how damp a fabric gets or how soaked an item is; the
moisture in the air or even the amount of water present in plants counts too. All these

##### Przykład czatu z użytkownikiem

Prosimy model o generację odpowiedzi i zapisujemy sobie historię interakcji do zmiennej `chat`, która jest listą słowników interakcji. Następnie dorzucamy do tej listy nowe zapytanie, na które odpowiedź ma być wygenerowana w kontekście całej konwersacji

In [7]:
messages = [
    {"role": "user", "content": "Is water wet? Elaborate"},
]
chat = pipe(messages, do_sample=True, temperature=0.01, max_new_tokens=100)[0]["generated_text"]
chat.append({"role": "user", "content": "What if we consider 'wetness' as an ability to make something wet? Is water still wet by that definition?"})
chat = pipe(chat, do_sample=True, temperature=0.01, max_new_tokens=50)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

Is water wet? Elaborate

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Water is indeed wet. The term "wet" refers to the presence of water or other liquids on a surface or in contact 
with an object. When water comes into contact with a surface, it adheres to it and can be absorbed by the material,
leading to a wet state.

Water is a polar molecule, meaning that it has a positive and a negative end. This polarity allows water molecules 
to form hydrogen bonds with other water molecules, which in

────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

What if we consider 'wetness' as an ability to make something wet? Is water still wet by that definition?

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Yes, by considering "wetness" as the ability to make something wet, water would still be considered wet. Water has
the ability to make other objects wet due to its properties, such as its polar nature and its ability to dissolve 
various

Kontynuujemy naszą rozmowę dorzucając kolejny element do naszej historii zawartej w zmiennej `chat`.

In [8]:
chat = chat[0]["generated_text"]
chat.append({"role": "user", "content": "Ok, what if an object is fully submersed? Is it still considered wet?"})
chat = pipe(chat, do_sample=True, temperature=2.0, max_new_tokens=50)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

Is water wet? Elaborate

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Water is indeed wet. The term "wet" refers to the presence of water or other liquids on a surface or in contact 
with an object. When water comes into contact with a surface, it adheres to it and can be absorbed by the material,
leading to a wet state.

Water is a polar molecule, meaning that it has a positive and a negative end. This polarity allows water molecules 
to form hydrogen bonds with other water molecules, which in

...

Yes, by considering "wetness" as the ability to make something wet, water would still be considered wet. Water has
the ability to make other objects wet due to its properties, such as its polar nature and its ability to dissolve 
various

────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

Ok, what if an object is fully submersed? Is it still considered wet?

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

When an object is fully submerged in water, any water that touches its surface and adheres to the material would, 
in turn, also considered wet due to becoming a part of the substance's surface or material, as well as its

# One-shot prompting

Pokazujemy modelowi przykład, a następnie dajemy mu zapytanie, na które powinien odpowiedzieć w analogiczny sposób. Na tym etapie nie mamy jeszcze explicite instrukcji w naszym prompcie.

In [9]:
one_shot_prompt = """
A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
 
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:
"""

messages = [
    {"role": "user", "content": one_shot_prompt},
]
chat = pipe(messages, do_sample=True, temperature=0.7, max_new_tokens=100)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
 
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle 
is:

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

After winning the game, the kids started farduddling with excitement.

# Few-shot prompting

Pokazujemy modelowi kilka przykładów i prosimy o uzupełnienie klasyfikacji ostatniego przykładu, ale nadal nie mamy explicite promptu z dokładnym poleceniem - model "domyśla się" o co nam chodzi na podstawie kontekstu.

In [10]:
few_shot_prompt = """
This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //
"""

messages = [
    {"role": "user", "content": few_shot_prompt},
]
chat = pipe(messages, do_sample=True, temperature=0.2, max_new_tokens=100)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Negative

This is a great book! // Positive
I can't believe I wasted my time on this! // Negative
I had so much fun at the party! // Positive
That concert was a total letdown! // Negative

Ten sam przykład ze zmienioną polaryzacją etykiet - teraz pozytywne zdania są negatywne, a negatywne pozytywne. Ponieważ nie podaliśmy konkretnych instrukcji model nie współpracuje i nie generuje etykiety dla ostatniego przykładu, który podaliśmy.

In [11]:
few_shot_prompt = """
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Negative
What a horrible show! //
"""

messages = [
    {"role": "user", "content": few_shot_prompt},
]
chat = pipe(messages, do_sample=True, temperature=0.2, max_new_tokens=100)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Negative
What a horrible show! //

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

This is a great tool! // Positive
I love this feature! // Positive
The new update is fantastic! // Positive
This is really useful! // Positive
I can't believe how easy this is to use! // Positive
I'm so impressed with the results! // Positive
This is a game-changer! // Positive
I can't wait to see more improvements! // Positive
This is a huge step forward!

##### Few-shot prompting + konkretna instrukcja

W prompcie typu `system` podajemy modelowi konkretną instrukcję, co pozwala mu na zrozumienie i wykonanie zadania.

In [12]:
few_shot_prompt = """
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Negative
What a horrible show! //
"""

messages = [
    {"role": "system", "content": "Your role is labelling sentences as either Negative or Positive. Based on user examples label the last sentence."},
    {"role": "user", "content": few_shot_prompt},
]
chat = pipe(messages, do_sample=True, temperature=0.2, max_new_tokens=100)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


───────────────────────────────────────────────────── system ──────────────────────────────────────────────────────

Your role is labelling sentences as either Negative or Positive. Based on user examples label the last sentence.

────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Negative
What a horrible show! //

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Positive

# Chain of Thought (refleksja)

Prosimy model o rozwiązanie przykładu z kijem i z piłeczką. W przypadku tej konkretnej sesji model sobie nie poradził, pomimo tego, że próbował opisywać krok po kroku swój tok rozumowania.

In [13]:
prompt = "A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost?"

messages = [
    {"role": "user", "content": prompt},
]
chat = pipe(messages, do_sample=True, temperature=0.2, max_new_tokens=200)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost?

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Let x be the cost of the ball. Then the cost of the bat is x + 1. According to the problem, the total cost of the 
bat and the ball is $1.10. So, we have the equation:

x + (x + 1) = 1.10

Combining like terms, we get:

2x = 1.10

Now, divide both sides by 2:

x = 0.55

So, the ball costs $0.55.

Dodajemy do promptu instrukcję `Let's think step by step`. W przypadku tej sesji wydaje się, że dodanie tej instrukcji pomogło modelowi rozwiązać problem, ale jak wspominałem na wykładzie efektywność metod refleksji w dużej mierze zależy od wielkości modelu (im większe tym więcej można ugrać za pomocą tych metod).

In [15]:
prompt = "A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost? Let's think step by step"

messages = [
    {"role": "user", "content": prompt},
]
chat = pipe(messages, do_sample=True, temperature=0.2, max_new_tokens=300)
pretty_print(chat)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


────────────────────────────────────────────────────── user ───────────────────────────────────────────────────────

A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost? Let's 
think step by step

──────────────────────────────────────────────────── assistant ────────────────────────────────────────────────────

Step 1: Let's represent the cost of the ball as "x". Then, the cost of the bat would be "x + $1.00".

Step 2: We know that the total cost of the bat and the ball is $1.10. So, we can write this as an equation:

(x + $1.00) + x = $1.10

Step 3: Now, we can simplify the equation by combining the terms with "x":

2x + $1.00 = $1.10

Step 4: Next, we can isolate "x" by subtracting $1.00 from both sides of the equation:

2x = $1.10 - $1.00
2x = $0.10

Step 5: Finally, we can solve for "x" by dividing both sides of the equation by 2:

x = $0.10 / 2
x = $0.05

So, the ball costs $0.05.